In [1]:
from bs4 import BeautifulSoup
import urllib.request
import requests
from pprint import pprint
import pandas as pd
import json
import copy
from datetime import datetime, timezone
from google.cloud import storage
import gcsfs
import re

In [2]:
storage_client = storage.Client(project="ytone-430507")
today = datetime.now()
table_name = "douyin_influencer"
api_name = "get_authors_ranking_in"

In [3]:
processing_blobs = [
{
    "blob": blob,
    "date": datetime.strptime(blob.name.split('/')[1], "%Y-%m-%d"),
    "batch_number": int(blob.name.split('/')[-1].replace(".json", "").split("_")[-1]),
} for blob in storage_client.list_blobs("0_raw_data",prefix="2_xingtu/") if api_name in blob.name]
processing_blobs

[{'blob': <Blob: 0_raw_data, 2_xingtu/2024-09-11/get_authors_ranking_in_0911_0.json, 1726037771193966>,
  'date': datetime.datetime(2024, 9, 11, 0, 0),
  'batch_number': 0},
 {'blob': <Blob: 0_raw_data, 2_xingtu/2024-09-13/get_authors_ranking_in_0913_0.json, 1726206085512338>,
  'date': datetime.datetime(2024, 9, 13, 0, 0),
  'batch_number': 0},
 {'blob': <Blob: 0_raw_data, 2_xingtu/2024-09-13/get_authors_ranking_in_0913_1.json, 1726206085980270>,
  'date': datetime.datetime(2024, 9, 13, 0, 0),
  'batch_number': 1},
 {'blob': <Blob: 0_raw_data, 2_xingtu/2024-09-13/get_authors_ranking_in_0913_10.json, 1726205577626093>,
  'date': datetime.datetime(2024, 9, 13, 0, 0),
  'batch_number': 10},
 {'blob': <Blob: 0_raw_data, 2_xingtu/2024-09-13/get_authors_ranking_in_0913_11.json, 1726205578089872>,
  'date': datetime.datetime(2024, 9, 13, 0, 0),
  'batch_number': 11},
 {'blob': <Blob: 0_raw_data, 2_xingtu/2024-09-13/get_authors_ranking_in_0913_12.json, 1726205578557746>,
  'date': datetime.da

In [4]:
processed_blobs = [{
    "blob": blob,
    "date": datetime.strptime(blob.name.split('/')[2], "%Y-%m-%d"),
    "batch_number": int(blob.name.split('/')[-1].replace(".parquet", "").split("_")[-1]),
} for blob in storage_client.list_blobs("3_staging_area",prefix="1_xingtu/") if table_name == blob.name.split("/")[1] and api_name in blob.name and blob.name.split("/")[2] and blob.name.split("/")[3]]
processed_blobs

[{'blob': <Blob: 3_staging_area, 1_xingtu/douyin_influencer/2024-09-11/get_authors_ranking_in_240923_0.parquet, 1727081498336366>,
  'date': datetime.datetime(2024, 9, 11, 0, 0),
  'batch_number': 0},
 {'blob': <Blob: 3_staging_area, 1_xingtu/douyin_influencer/2024-09-13/get_authors_ranking_in_240919_0.parquet, 1726733100301993>,
  'date': datetime.datetime(2024, 9, 13, 0, 0),
  'batch_number': 0},
 {'blob': <Blob: 3_staging_area, 1_xingtu/douyin_influencer/2024-09-13/get_authors_ranking_in_240919_1.parquet, 1726733101898156>,
  'date': datetime.datetime(2024, 9, 13, 0, 0),
  'batch_number': 1},
 {'blob': <Blob: 3_staging_area, 1_xingtu/douyin_influencer/2024-09-13/get_authors_ranking_in_240919_10.parquet, 1726733103464486>,
  'date': datetime.datetime(2024, 9, 13, 0, 0),
  'batch_number': 10},
 {'blob': <Blob: 3_staging_area, 1_xingtu/douyin_influencer/2024-09-13/get_authors_ranking_in_240919_11.parquet, 1726733105012034>,
  'date': datetime.datetime(2024, 9, 13, 0, 0),
  'batch_numbe

In [5]:
to_process = []
for processing_blob in processing_blobs:
    if processing_blob["date"] >= datetime(2024, 9, 13):
        if processing_blob["batch_number"] not in [processed_blob["batch_number"] for processed_blob in processed_blobs if processing_blob["date"] == processed_blob["date"]]:
            to_process.append(processing_blob)
pprint(to_process)
print(len(to_process))

[{'batch_number': 0,
  'blob': <Blob: 0_raw_data, 2_xingtu/2024-10-11/get_authors_ranking_in_1011_0.json, 1728633459585513>,
  'date': datetime.datetime(2024, 10, 11, 0, 0)},
 {'batch_number': 1,
  'blob': <Blob: 0_raw_data, 2_xingtu/2024-10-11/get_authors_ranking_in_1011_1.json, 1728633459984648>,
  'date': datetime.datetime(2024, 10, 11, 0, 0)},
 {'batch_number': 2,
  'blob': <Blob: 0_raw_data, 2_xingtu/2024-10-11/get_authors_ranking_in_1011_2.json, 1728633460407857>,
  'date': datetime.datetime(2024, 10, 11, 0, 0)},
 {'batch_number': 3,
  'blob': <Blob: 0_raw_data, 2_xingtu/2024-10-11/get_authors_ranking_in_1011_3.json, 1728633460771973>,
  'date': datetime.datetime(2024, 10, 11, 0, 0)},
 {'batch_number': 4,
  'blob': <Blob: 0_raw_data, 2_xingtu/2024-10-11/get_authors_ranking_in_1011_4.json, 1728633461121244>,
  'date': datetime.datetime(2024, 10, 11, 0, 0)},
 {'batch_number': 5,
  'blob': <Blob: 0_raw_data, 2_xingtu/2024-10-11/get_authors_ranking_in_1011_5.json, 1728633461470012>,


In [6]:
def get_authors_ranking_in_data(raw_data):
    result = []
    for data in raw_data:
        profile_id = data["profile_id"]
        get_authors_ranking_in = data.get("get_authors_ranking_in")
        if get_authors_ranking_in:
            item = get_authors_ranking_in[0]
            record = {}
            record["level"] = item["level"]
            record["user_id"] = profile_id
            result.append(record)
    return result

In [7]:
for item in to_process:
    raw_data = json.loads(item["blob"].download_as_string())
    authors_ranking_in_data = get_authors_ranking_in_data(raw_data)
    df = pd.DataFrame(authors_ranking_in_data)
    if len(df):
        df.level = df.level.astype(int)
        df.to_parquet("gs://3_staging_area/1_xingtu/douyin_influencer/" + str(item["date"].date()) + "/get_authors_ranking_in_" + today.strftime("%y%m%d") + "_" + str(item["batch_number"]) + ".parquet")